#  Yahoo News Scraper

In [1]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

In [2]:
template = 'https://in.search.yahoo.com/search?p={}'

In [3]:
url = template.format('iphone 12 leaked')

In [6]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
    'referer': 'https://in.news.yahoo.com/',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36'
}

In [7]:
response = requests.get(url, headers = headers)

In [9]:
soup = BeautifulSoup(response.text, 'html.parser')

GET THE COLLECTION

In [19]:
cards = soup.find_all('section', 'algo')

In [20]:
len(cards)

8

In [21]:
cards

[<section class="dd algo s-algo algo-sr fst lst Sr" data-428="5fe96c00969d0"><div class="grp grp-talgo ag-1 u-tapHgt u-negmlrEdges u-plrEdges bb-1 cur-p"><div class="compTitle options-toggle"><h3 class="title"><a class="s-title fz-m" data-428="5fe96c0096ab4" href="https://r.search.yahoo.com/_ylt=AwrPhO8AbOlfrjYAQBjZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1609161856/RO=10/RU=https%3a%2f%2fwww.moneycontrol.com%2fnews%2ftechnology%2fapple-iphone-12-launch-prices-leaked-before-october-launch-here-are-all-the-details-5919061.html/RK=2/RS=NV5MoX03qKN_cIU1Kxkd9tuBilU-" referrerpolicy="origin">Apple IPhone 12 Prices Leaked Before October Launch; Here Are ...</a></h3> <div><span class="s-source s-url cite d-b tc w-100p lh-20"><b>www.moneycontrol.com</b>/news/technology/apple...</span></div></div></div><div class="grp grp-talgo-ext u-negmlrEdges u-plrEdges"><div class="compText"><p class="s-desc lh-20 fbox-lc3 d-box ov-h fbox-ov">Apple <b>iPhone 12 </b>prices <b>leaked </b>before 

Create a prototype model

In [22]:
card = cards[0]

In [26]:
headline= card.find('a', 's-title').text

In [27]:
headline

'Apple IPhone 12 Prices Leaked Before October Launch; Here Are ...'

In [28]:
source = card.find('span', 's-source').text

In [29]:
source

'www.moneycontrol.com/news/technology/apple...'

In [30]:
description = card.find('p', 's-desc').text

In [31]:
description

'Apple iPhone 12 prices leaked before October launch; here are all the details The smallest rumoured version, the iPhone 12 mini may start at $649 (Rs 47,600) while the... '

In [32]:
raw_link = card.find('a').get('href')
raw_link

'https://r.search.yahoo.com/_ylt=AwrPhO8AbOlfrjYAQBjZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1609161856/RO=10/RU=https%3a%2f%2fwww.moneycontrol.com%2fnews%2ftechnology%2fapple-iphone-12-launch-prices-leaked-before-october-launch-here-are-all-the-details-5919061.html/RK=2/RS=NV5MoX03qKN_cIU1Kxkd9tuBilU-'

In [33]:
unquoted_link = requests.utils.unquote(raw_link)
unquoted_link

'https://r.search.yahoo.com/_ylt=AwrPhO8AbOlfrjYAQBjZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1609161856/RO=10/RU=https://www.moneycontrol.com/news/technology/apple-iphone-12-launch-prices-leaked-before-october-launch-here-are-all-the-details-5919061.html/RK=2/RS=NV5MoX03qKN_cIU1Kxkd9tuBilU-'

In [34]:
pattern = re.compile(r'RU=(.+)\/RK')
clean_link = re.search(pattern, unquoted_link).group(1)
clean_link

'https://www.moneycontrol.com/news/technology/apple-iphone-12-launch-prices-leaked-before-october-launch-here-are-all-the-details-5919061.html'

Generalize the model

In [35]:
def get_article(card):
    """Extract article information from the raw html"""
    headline= card.find('a', 's-title').text
    source = card.find('span', 's-source').text
    description = card.find('p', 's-desc').text
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clean_link = re.search(pattern, unquoted_link).group(1)
    
    article = (headline, source, description, clean_link)
    return article

In [36]:
articles = []
links = set()

for card in cards:
    article = get_article(card)
    link = article[-1]
    if not link in links:
        links.add(link)
        articles.append(article)

In [37]:
articles[:5]

[('Apple IPhone 12 Prices Leaked Before October Launch; Here Are ...',
  'www.moneycontrol.com/news/technology/apple...',
  'Apple iPhone 12 prices leaked before October launch; here are all the details The smallest rumoured version, the iPhone 12 mini may start at $649 (Rs 47,600) while the... ',
  'https://www.moneycontrol.com/news/technology/apple-iphone-12-launch-prices-leaked-before-october-launch-here-are-all-the-details-5919061.html'),
 ('iPhone 12 Pro leaked in new hands-on video: Watch video ...',
  'www.indiatvnews.com/technology/news-apple-iphone...',
  "Leaked iPhone 12 Pro Apple is quite near to the unveiling of the 2020 iPhone 12 lineup, which could either arrive next month or at the Cupertino major's September 15... ",
  'https://www.indiatvnews.com/technology/news-apple-iphone-12-pro-leaked-hands-on-video-lidar-sensor-flat-edges-and-more-648995'),
 ('iPhone 12 leaked dummy units hint at three screen ... - India TV',
  'www.indiatvnews.com/technology/news-apple-iphone...

Get the next page

In [40]:
url = soup.find('a', 'next').get('href')

In [41]:
url

'https://in.search.yahoo.com/search;_ylt=AwrPhO8AbOlfrjYAZBjZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3BhZ2luYXRpb24-?p=iphone+12+leaked&b=11&pz=10&pstart=4'

Bringing it all together

In [52]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
    'referer': 'https://in.news.yahoo.com/',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline= card.find('a', 's-title').text
    source = card.find('span', 's-source').text
    description = card.find('p', 's-desc').text
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clean_link = re.search(pattern, unquoted_link).group(1)
    
    article = (headline, source, description, clean_link)
    return article

def get_the_news(search):
    """Run the main program"""
    template = 'https://in.search.yahoo.com/search?p={}'
    url = template.format(search)
    articles = []
    links = set()
    
    while True:
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('section', 'algo')
        
        # extract articles from the page
        for card in cards:
            article = get_article(card)
            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)
        # find the next page
        try:
            url = soup.find('a','next').get('href')
            print(url)
            sleep(1)
        except AttributeError:
            break
            
    # save articles data
    with open('yahoo-news.csv','w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Headline', 'Source', 'Description', 'Link'])
        writer.writerows(articles)
            
    return articles

In [53]:
results = get_the_news("iphone 12 leaked")

https://in.search.yahoo.com/search;_ylt=AwrxgKA_fOlf0isA_RzZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3BhZ2luYXRpb24-?p=iphone+12+leaked&b=11&pz=10&pstart=5
https://in.search.yahoo.com/search;_ylt=AwrxiDdAfOlfjxIA0gLZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3BhZ2luYXRpb24-?p=iphone+12+leaked&pz=10&b=21&pz=10&pstart=6
https://in.search.yahoo.com/search;_ylt=AwrwZeVCfOlfiD0ADhfZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3BhZ2luYXRpb24-?p=iphone+12+leaked&pz=10&b=31&pz=10&pstart=5
https://in.search.yahoo.com/search;_ylt=AwrPgxFEfOlfzH0A5CbZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3BhZ2luYXRpb24-?p=iphone+12+leaked&pz=10&b=41&pz=10&pstart=5
https://in.search.yahoo.com/search;_ylt=AwrPgxBFfOlfaigAKC_ZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3BhZ2luYXRpb24-?p=iphone+12+leaked&pz=10&b=51&pz=10&pstart=5
https://in.search.yahoo.com/search;_ylt=AwrwS5lHfOlfdgUA1QPZjSM5;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3BhZ2luYXRpb24-?p=iphone+12+leaked&pz=10&b=61&pz=10&xargs=0
https://in.search.yahoo